# Recommendations Using Hybrid solutions <sup>1</sup>

Lots of examples of systems that try to combine the strengths of the two main approaches.
This can be done in a number of ways:

- Combine the predictions of a content-based system and a collaborative system.
- Incorporate content-based techniques into a collaborative approach.
- Incorporate collaborative techniques into a content-based approach.
- Unifying model.

## Imports

In [1]:
import numpy as np
import pandas as pd
from scipy.special import logsumexp

## Load Data

Only loading a subset of the original data set for proof of concept reasons.

In [2]:
# 80/20 split earlier
df_train = pd.read_csv('../Data/training_data_subset.csv')
df_test = pd.read_csv('../Data/testing_data_subset.csv')

In [3]:
df_test.head(2)

,category,title,also_buy,brand,rank,also_view,main_cat,price,asin,overall,verified,reviewerID,vote,style,for_testing
0,"['Grocery & Gourmet Food', 'Snack Foods', 'Bar...","Grocery &amp; Gourmet Food"" />","['B01MT0QDPO', 'B00NL17FE4', 'B01NBM9OJN', 'B0...",Nature Valley,"16,921 in Grocery & Gourmet Food (",NaN,Grocery,18.04,B001E6GFR6,5.0,True,A2IUE299OONA73,NaN,NaN,True
1,"['Grocery & Gourmet Food', 'Snack Foods', 'Chi...",Gourmet Basics Smart Fries 4-Flavor Variety Pa...,"['B0763SHX4W', 'B0040FIHS8', 'B00FYR5HS4', 'B0...",Gourmet Basics,"53,167 in Grocery & Gourmet Food (",NaN,Grocery,23.99,B003AZ2ECY,4.0,True,A38NO7J1TK4R1W,NaN,NaN,True


In [4]:
df_train.head(2)

,category,title,also_buy,brand,rank,also_view,main_cat,price,asin,overall,verified,reviewerID,vote,style,for_testing
0,"['Grocery & Gourmet Food', 'Candy & Chocolate'...","YumEarth Organic Gummy Bears, 10 Count","['B008CC8UXC', 'B00C25LO8S', 'B073RWDCMD', 'B0...",YumEarth,"129,438 in Grocery & Gourmet Food (","['B008CC8UXC', 'B00C25LNWA', 'B008CC8ULY', 'B0...",Grocery,NaN,B008B7JNRA,3.0,True,A35KP4ROS9KWPO,NaN,"{'Size:': ' 10 Count', 'Style:': ' Natural Gum...",False
1,"['Grocery & Gourmet Food', 'Jams, Jellies & Sw...",Bell Plantation Powdered PB2 Bundle: 1 Peanut ...,"['B06W9N8X9H', 'B06X15V3DC', 'B01ENYJX3S', 'B0...",PB2,"1,214 in Grocery & Gourmet Food (",NaN,Grocery,18.49,B00H9H56QA,5.0,True,AVAMZWS7AAI1S,NaN,{'Size:': ' Pack of 2 (1 each flavor)'},False


### Evaluate

In [9]:
def compute_rmse(y_pred, y_true):
    """ Compute Root Mean Squared Error. """
    
    return np.sqrt(np.mean(np.power(y_pred - y_true, 2)))

def evaluate(estimate_f):
    """ RMSE-based predictive performance evaluation with pandas. """
    
    ids_to_estimate = zip(df_train.reviewerID, df_train.asin)
    estimated = np.array([estimate_f(u,i) for (u,i) in ids_to_estimate])
    real = df_train.overall.values
    return compute_rmse(estimated, real)

In [10]:
def evaluate_test(estimate_f):
    """ RMSE-based predictive performance evaluation with pandas. """
    
    ids_to_estimate = zip(df_test.reviewerID, df_test.asin)
    estimated = np.array([estimate_f(u,i) for (u,i) in ids_to_estimate])
    real = df_test.overall.values
    return compute_rmse(estimated, real)

## Similarity functions

In [11]:
def euclidean(s1, s2):
    """Take two pd.Series objects and return their euclidean 'similarity'."""
    diff = s1 - s2
    return 1 / (1 + np.sqrt(np.sum(diff ** 2)))

def cosine(s1, s2):
    """Take two pd.Series objects and return their cosine similarity."""
    return np.sum(s1 * s2) / np.sqrt(np.sum(s1 ** 2) * np.sum(s2 ** 2))

def pearson(s1, s2):
    """Take two pd.Series objects and return a pearson correlation."""
    s1_c = s1 - s1.mean()
    s2_c = s2 - s2.mean()
    return np.sum(s1_c * s2_c) / np.sqrt(np.sum(logsumexp(s1_c ** 2)) * np.sum(logsumexp(s2_c ** 2)))

def jaccard(s1, s2):
    dotp = np.sum(s1 * s2)
    return dotp / (np.sum(s1 ** 2) + np.sum(s2 ** 2) - dotp)

In [12]:
class Recommender(object):
    def __init__(self, similarity=pearson):
        self.overall_mean = df_train['overall'].mean()
        self.all_user_profiles = df_train.pivot_table('overall', index='asin', columns='reviewerID')
        self._similarity = similarity
        
    @property
    def similarity(self):
        return self._similarity
    
    @similarity.setter
    def similarity(self, value):
        self._similarity = value
    
    def estimate_product(self, user_id, product_id):
        all_ratings = df_train.loc[df_train.asin == product_id]
        if all_ratings.empty:
            return self.overall_mean
        all_ratings.set_index('reviewerID', inplace=True)
        their_ids = all_ratings.index
        their_ratings = all_ratings.overall
        their_profiles = self.all_user_profiles[their_ids]
        user_profile = self.all_user_profiles[user_id]
        sims = their_profiles.apply(lambda profile: self.similarity(profile, user_profile), axis=0)
        ratings_sims = pd.DataFrame({'sim': sims, 'overall': their_ratings})
        ratings_sims = ratings_sims[ratings_sims.sim > 0]
        if ratings_sims.empty:
            return their_ratings.mean()
        else:
            return np.average(ratings_sims.overall, weights=ratings_sims.sim)

    def estimate_user(self, user_id):
        similar = df_train.loc[df_train.reviewerID == user_id]
        
        if similar.empty:
            return self.overall_mean
        else:
            return similar['overall'].mean()

    def estimate(self, user_id, movie_id):
        return 0.9 * self.estimate_user(user_id) + 0.1 * self.estimate_product(user_id, movie_id)

In [21]:
rec = Recommender(euclidean)
print('RMSE using euclidean: %s' % evaluate(rec.estimate))

RMSE using euclidean: 0.2026571693473235


In [22]:
rec = Recommender(cosine)
print('RMSE using cosine: %s' % evaluate(rec.estimate))

RMSE using cosine: 0.2076234970442699


In [23]:
rec = Recommender(pearson)
print('RMSE using pearson: %s' % evaluate(rec.estimate))

RMSE using pearson: 0.21002381594394504


In [24]:
rec = Recommender(jaccard)
print('RMSE using jaccard: %s' % evaluate(rec.estimate))

RMSE using jaccard: 0.20355713315116886


## Summary
- Evaluated estimated recommendations using several hybrid methods.
- In all cases, the estimates were significantly better than all previous notebook versions.
- The best performing was euclidean with a RMSE of 0.2027.
- The worse performing was pearson with a RMSE of 0.21.

## References
1) Unata 2015 [Hands-on with PyData: How to Build a Minimal Recommendation Engine](https://www.youtube.com/watch?v=F6gWjOc1FUs).  